In [19]:
import glob

import mne

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

## Data Asli

In [21]:
raw = mne.io.read_raw_edf("/home/fahmi/research/eegautism/data/edf/autism/Bader_Autism_24_11_2011S001R08.edf")
raw

Extracting EDF parameters from /home/fahmi/research/eegautism/data/edf/autism/Bader_Autism_24_11_2011S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<RawEDF | Bader_Autism_24_11_2011S001R08.edf, 16 x 18096 (70.7 s), ~17 kB, data not loaded>

1.1780496000000003

In [41]:
raw_samples = []
for file_path in glob.glob("../data/edf/**/*.edf"):
    raw = mne.io.read_raw_edf(file_path, verbose=False)
    file_name = file_path.split("/")[-1].lower().split(".")[0]

    raw_samples.append({
        "patient_name": file_name.split("_")[0],
        "duration_sec": raw.n_times / raw.info["sfreq"],
        "recording_number": int(file_name[-2:]),
        "label": 1 if "autism" in file_name else 0
    })
    
df_raw = pd.DataFrame(raw_samples)
df_raw["duration"] = pd.to_timedelta(df_raw["duration_sec"], unit="s")
df_raw.head()

,patient_name,duration_sec,recording_number,label,duration
0,amer,317.323440,3,0,0 days 00:05:17.323440
1,majdolin,955.345104,1,0,0 days 00:15:55.345104
2,mahmud,455.345856,1,0,0 days 00:07:35.345856
3,dhelal,301.480704,1,0,0 days 00:05:01.480704
4,dhelal,901.473552,2,0,0 days 00:15:01.473552


In [42]:
df_raw.groupby("label").sum()

,patient_name,duration_sec,recording_number,duration
label,,,,
0,amermajdolinmahmuddhelaldhelalamermahmudomranamer,4851.939456,14,0 days 01:20:51.939456
1,yahiabaderbadernourbaderduaaomarshahadsaudyahi...,10572.229584,58,0 days 02:56:12.229584


In [50]:
df_raw.groupby(["label", "patient_name"]).max()[["recording_number", "duration"]]

recording_number               duration
label patient_name                                         
0     amer                         3 0 days 00:06:06.882768
      dhelal                       2 0 days 00:15:01.473552
      mahmud                       2 0 days 00:07:35.345856
      majdolin                     1 0 days 00:15:55.345104
      omran                        1 0 days 00:15:02.848464
1     bader                       10 0 days 00:13:48.634464
      deena                        1 0 days 00:02:55.363776
      duaa                         1 0 days 00:07:00.848064
      mada                         2 0 days 00:14:25.663344
      mohammed                     1 0 days 00:18:27.116640
      nour                         3 0 days 00:12:45.638496
      omar                         1 0 days 00:40:24.969792
      saud                         1 0 days 00:12:30.326976
      shahad                       1 0 days 00:20:30.015024
      yahia                        2 0 days 00:12:04.672368

## Data Praproses

In [2]:
ds = xr.open_dataset("../data/data_10.nc")
ds

<xarray.Dataset> Size: 220MB
Dimensions:           (samples: 670, time_steps: 2560, channels: 16)
Coordinates:
  * samples           (samples) int64 5kB 0 1 2 3 4 5 ... 665 666 667 668 669
Dimensions without coordinates: time_steps, channels
Data variables:
    signal            (samples, time_steps, channels) float64 220MB ...
    label             (samples) int64 5kB ...
    segment           (samples) int64 5kB ...
    patient_name      (samples) <U8 21kB ...
    recording_number  (samples) int64 5kB ...

In [62]:
df_proc = ds[["label", "segment", "patient_name", "recording_number"]].to_dataframe()
df_proc.head()

,label,segment,patient_name,recording_number
samples,,,,
0,1,0,yahia,1
1,1,1,yahia,1
2,1,2,yahia,1
3,1,3,yahia,1
4,1,4,yahia,1


In [82]:
df_procseg

0
label patient_name    
0     amer          85
      dhelal        84
      mahmud        44
      omran         64
1     bader         64
      mohammed      99
      nour          71
      saud          46
      shahad        38
      yahia         75

In [85]:
df_procseg = df_proc.groupby(["label", "patient_name"]).size().to_frame().rename(columns={0: "segment"})
df_procseg["duration"] = pd.to_timedelta(df_procseg["segment"] * 10, unit="s")
df_procseg

segment        duration
label patient_name                         
0     amer               85 0 days 00:14:10
      dhelal             84 0 days 00:14:00
      mahmud             44 0 days 00:07:20
      omran              64 0 days 00:10:40
1     bader              64 0 days 00:10:40
      mohammed           99 0 days 00:16:30
      nour               71 0 days 00:11:50
      saud               46 0 days 00:07:40
      shahad             38 0 days 00:06:20
      yahia              75 0 days 00:12:30

In [68]:
df_proc["label"].value_counts()

label
1    393
0    277
Name: count, dtype: int64

In [88]:
df_procseg.reset_index().groupby("label").sum()

,patient_name,segment,duration
label,,,
0,amerdhelalmahmudomran,277,0 days 00:46:10
1,badermohammednoursaudshahadyahia,393,0 days 01:05:30
